# SMFI Microscope Control GUI
Ben Jones 2022 (ben.jones@uta.edu)

In [1]:
from GUI import MicroGui

In [2]:
# Set this flag to true if you're running on the actual microscope computer
HARDWARE_ON=False

# Determine whether to use image compression
COMPRESSION_ON=False

if(HARDWARE_ON):
    from Stage import Stage
    import CamHamPy


In [ ]:
MicroGui(HARDWARE_ON, COMPRESSION_ON)

No lastpos file loaded, initializing to defaults


/Users/jonesb/Documents/Work/AutoF/MicroscopeControlSuite/GUI.py:380: UserWarning: Attempting to set identical bottom == top == 65476.84452249071 results in singular transformations; automatically expanding.
  self.ax_z[0].set_ylim(BottomIntensity,TopIntensity)
/Users/jonesb/Documents/Work/AutoF/MicroscopeControlSuite/GUI.py:381: UserWarning: Attempting to set identical bottom == top == -1.2006459522258817 results in singular transformations; automatically expanding.
  self.ax_z[1].set_ylim(BottomKurtosis,TopKurtosis)

KeyboardInterrupt



KeyboardInterrupt: 